In [ ]:
from AutoEncoders import TopKSAE, JumpReLUSAE, ReLUSAE, OrthogonalSAE



from Utils import createImageDataloader
import clip
import torch, torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import torch, torch.nn as nn, torch.optim as optim
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
DATA_DIR = '/Users/mawy/Desktop/Square Dataset/imgs'
W, H = 224, 224
BATCH = 64

loader = createImageDataloader(DATA_DIR, W, H, bsize=BATCH, shuffle=True)
xb, _ = next(iter(loader))
print(xb.shape)

torch.Size([64, 3, 224, 224])


In [4]:
vit, preprocess = clip.load('ViT-B/32', device=device)
vit.eval();

In [ ]:
sae = ReLUSAE(d=512, n=128, lam=0.1)
sae.findBias(vit, loader, 1)

[LOG] Bias initialized to Geometric Median


In [5]:
epochs = 1
lr = 1e-3

optimizer = optim.Adam(sae.parameters(), lr=lr)
criterion = nn.MSELoss()

for epoch in range(1, epochs + 1):
    sae.train()
    running_loss = 0.0
    pbar = tqdm(loader, desc=f"Epoch {epoch}/{epochs}", unit="batch")

    for xb, _ in pbar:
        xb = xb.to(device)

        with torch.no_grad():
            feats = vit.visual(xb)      # [B, 768]

        optimizer.zero_grad()
        recon, z = sae(feats)           # recon: [B,768]
        loss = criterion(recon, feats)

        loss.backward()
        optimizer.step()
        sae.renorm()

        running_loss += loss.item() * xb.size(0)
        pbar.set_postfix(batch_loss=loss.item(), avg_loss=(running_loss / ((pbar.n+1)*loader.batch_size)))

    avg_loss = running_loss / len(loader.dataset)
    print(f"Epoch {epoch}/{epochs} finished — Avg Reconstruction Loss: {avg_loss:.4f}")

Epoch 1/1:   0%|          | 0/250 [00:15<?, ?batch/s]


ValueError: too many values to unpack (expected 2)